In [ ]:
import os
import csv
import time
import pickle
import numpy as np
import pandas as pd
from tqdm import tqdm
import nltk

## Deal with ocr data

In [ ]:
df_ocr=pd.read_csv("../dataset/rawdata/ocr.csv")
df_ocr=df_ocr.drop(columns=['humor','com_score'])
df_ocr.rename(columns={'U':'User','Humorous_reply':'Humorous reply'},inplace = True)
df_ocr

## Deal with train345k

In [ ]:
df_160=pd.read_csv("../dataset/rawdata/train160k.csv")
df_160=df_160[df_160['humor']==True]
df_160=df_160[df_160['text'].apply(lambda x: len(nltk.sent_tokenize(x))==2)]
df_160

In [ ]:
df_345=pd.read_csv("../dataset/rawdata/train345k.tsv",header=None,usecols=[0,4], names=['text', 'humor'])
df_345['text']=df_345['text'].apply(lambda x:x.split('\t')[1])
df_345=df_345[df_345['text'].apply(lambda x: len(nltk.sent_tokenize(x))==2)]
df_345

In [ ]:
df_all=df_160.append(df_345)
df_all=df_all.drop_duplicates(subset=['text'])
df_all

In [ ]:
df_all['User']=" "
df_all['Humorous reply']=" "
df_all['User']=df_all['text'].apply(lambda x:nltk.sent_tokenize(x)[0])
df_all['Humorous reply']=df_all['text'].apply(lambda x:nltk.sent_tokenize(x)[1])

In [48]:
df_all=df_all.drop(columns=['text','humor'])

In [50]:
df_all=df_all.append(df_ocr)
df_all

,User,Humorous reply
2,What do you call a turtle without its shell?,dead.
6,What is a pokemon master's favorite kind of pa...,wartortellini!
7,Why do native americans hate it when it rains ...,because it brings mayflowers.
13,How are music and candy similar?,we throw away the rappers.
21,Dating tip: surprise your date!,show up a day early.
...,...,...
311,I’m in a lot of pain and could use some light ...,Do you know the one about the blind hooker?
312,I’m going out tomorrow night with some friends...,new ice age?
313,Why do you pick someone up off the ground when...,i guess i get a little carried away.
314,What's your unique talent?,"I've had people start looking around, trying t..."


In [52]:
df_all=df_all.drop_duplicates(subset=['User'])
df_all['User']=df_all['User'].apply(lambda x:"User: "+x)
df_all['Humorous reply']=df_all['Humorous reply'].apply(lambda x:"Humorous reply: "+x)

In [76]:
df_all = df_all.sample(frac=1).reset_index(drop=True)
df_all

,User,Humorous reply
0,User: Looking at cleavage is like looking at t...,Humorous reply: You look
1,User: How warm is it inside a Tauntaun?,Humorous reply: Just Hoth freezing
2,User: A woman arrived at a party & while scann...,Humorous reply: She approached him smiled and ...
3,User: Every year John entered the state lotter...,Humorous reply: Finally he prayed vigorously a...
4,User: Heard the sperm bank gives $50 for your ...,Humorous reply: I have a sock in my room worth...
...,...,...
198371,User: Hey girl are you lava?,Humorous reply: Because I am sure not falling ...
198372,User: Why did Saudi Arabia banned chess and ca...,Humorous reply: Because
198373,User: What's the most important job?,Humorous reply: A chemist
198374,User: What is an alcoholic's favorite type of ...,Humorous reply: 12 bar blues.


In [77]:
my_data=df_all
train_size = 0.7
validate_size = 0.2
train, validate, test = np.split(my_data.sample(frac=1), [int(train_size * len(my_data)), int((validate_size + train_size) * len(my_data))])

In [78]:
train

,User,Humorous reply
91475,User: Did you hear about the blind man who was...,Humorous reply: He said it was the most violen...
117911,User: What would you call Hitler if he lost an...,Humorous reply: Htler
134875,User: My least favorite video game character i...,Humorous reply: ...Because he sucks.
34253,User: What do you call a vegetarian that occas...,Humorous reply: A liar
40055,User: I set all the cattle free.,Humorous reply: - reverse cowgirl
...,...,...
65556,User: If Cats Never Existed... Then Donald Tru...,Humorous reply: It would be like reaching out ...
167995,User: Mate of mine has two tickets for the Eur...,Humorous reply: If you are interested and want...
160550,User: What does Dr Who eat with pasta?,Humorous reply: Dalek Bread!
151109,User: A squad of British soldiers is stuck on ...,Humorous reply: (This does not belong to me


In [79]:
validate

,User,Humorous reply
88940,User: I suffer from a noxious stimulus down my...,Humorous reply: It's a pain in the ass.
22749,User: What do you call a hot girl with no legs?,Humorous reply: disabled.
62683,User: Ever read the fanfic crossover about Oed...,Humorous reply: I hear it's motherfucking gold.
33426,User: Do you know what the Irish do really well?,Humorous reply: Sheep
65388,User: What do chefs and murderers have in common?,Humorous reply: the best ones clean up after t...
...,...,...
67625,User: A man is on his way home after a busines...,Humorous reply: While being distracted for a m...
59697,User: A goldfish flops into a bar.,Humorous reply: The bartender is surprised to ...
141023,User: What do you call people who don't care a...,Humorous reply: Liars.
120878,User: I love the smell of my F5 key.,Humorous reply: It's so refreshing.


In [80]:
test

,User,Humorous reply
57611,User: What kind of bovine is always behind?,Humorous reply: A butt ox
185317,User: How do you turn a duck into a soul singer!,Humorous reply: Put it in the oven at 350 unti...
138701,User: Did you know that non vaxxed kids had su...,Humorous reply: Their bodies are way more sens...
81530,User: What do you call a horse with magic powers?,Humorous reply: Harry Trotter.
167911,User: What did the reddish poo say to the brow...,"Humorous reply: ""I feel flushed."""
...,...,...
124666,User: What's the contour integral of western E...,Humorous reply: Zero
101228,User: Which kitchen appliance do surfers disli...,Humorous reply: the micro wave.
24271,User: Did you hear about the band the hipster ...,"Humorous reply: no, no you didn't."
195589,User: I've never heard a good circumcision joke.,Humorous reply: They always get cut off right ...


In [81]:
file1 = open("../dataset/train/train.txt","w")
for index, row in train.iterrows():
    str1=row['User']
    str2=row['Humorous reply']
    file1.write(str1)
    file1.write('\n')
    file1.write(str2)
    file1.write('<|endoftext|>')
file1.close

<function TextIOWrapper.close()>

In [82]:
file2 = open("../dataset/val/val.txt","w")
for index, row in validate.iterrows():
    str1=row['User']
    str2=row['Humorous reply']
    file2.write(str1)
    file2.write('\n')
    file2.write(str2)
    file2.write('<|endoftext|>')
file2.close

<function TextIOWrapper.close()>

In [83]:
file3 = open("../dataset/test/test.txt","w")
for index, row in test.iterrows():
    str1=row['User']
    str2=row['Humorous reply']
    file3.write(str1)
    file3.write('\n')
    file3.write(str2)
    file3.write('<|endoftext|>')
file3.close

<function TextIOWrapper.close()>

In [84]:
file4 = open("../dataset/test_user.txt","w")
for index, row in test.iterrows():
    str1=row['User']
#     str2=row['Humorous reply']
    file4.write(str1)
    file4.write('\n')
#     file4.write('Humorous reply: ')
file4.close


<function TextIOWrapper.close()>